In [18]:
import pygame
import sys

#definerer fargene grønn hvit og rød
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)


#starter pygame og setter størrelsen på vindu 
pygame.init()
overflate = pygame.display.set_mode((612, 408))
pygame.display.set_caption("menu") #setter navn på vindu 

#definerer variabler for spill menuen  
bg = pygame.image.load("Pingpongmenu.png") #setter bakgrunds bilde 
menu_title = pygame.image.load("menutitle.png")
play_knapp = pygame.image.load("playknapp.png")
help_knapp = pygame.image.load("helpknapp.png")
close_knapp = pygame.image.load("closeknapp.png")

#definerer rektangler på samme sted som de tidligere variablene  
menu_title_rect = menu_title.get_rect(topleft=(130, 8))
play_knapp_rect = play_knapp.get_rect(topleft=(215, 108))
help_knapp_rect = help_knapp.get_rect(topleft=(215, 208))
close_knapp_rect = close_knapp.get_rect(topleft=(215, 308))

menu = True

#kjekker om musen har truffet en rektangel 
def Museklikk(posisjon, rektangel): 
    return rektangel.collidepoint(posisjon)

#koden for å skrive tekst 
def skriv(s, stor, x, y, farge):
    font = pygame.font.Font(None, stor)
    tekstflate = font.render(s, True, farge)
    overflate.blit(tekstflate, (x, y))

#selve spill funksjonen
def Play():
    pygame.display.set_caption("Spill") # setter navn på spill vindu
    
    #definerer raketen spilleren styrer, og dens posisjon 
    spiller_objekt = pygame.image.load("Bordtennisracket.png")
    spiller_objektx = 10
    spiller_objekty = 150
    
    #definerer en knap for å returnere til menuen 
    menu_knapp = pygame.image.load("menuknapp.png") 
    menu_knappx = 10
    menu_knappy = 10
    menu_knapp_rect = menu_knapp.get_rect(topleft=(10, 10)) #her etablerer den en raktangel i samme sted
    
    racket_topp_grense = 0
    racket_bunn_grense = 308
    
    #definerer motstanderens racket og gir den en visuell bilde 
    ikke_spiller_kølle = pygame.image.load("Grønn ping pong kølle.png")
    ikke_spiller_køllex = 522
    ikke_spiller_kølley = 150
    
    #definerer selve pingpongballen og gir den også et bilde 
    Pingpongball = pygame.image.load("pingpongball.png")
    Pingpongballx = 300
    Pingpongbally = 204

    y_akse = 2
    x_akse = 5
    ball_y_akse = 5
    clock = pygame.time.Clock() #definerer en variabel som styrer tiden på programmet 

    kjøre = True
    poeng = 0
    poeng_tid = 0 
    
    while kjøre == True:
        #plasserer alle variablene inn i pygame slik at de kan bli sett i spilet
        overflate.blit(pygame.image.load("PingPong.png"), (0, 0))
        overflate.blit(spiller_objekt, (spiller_objektx, spiller_objekty))
        overflate.blit(ikke_spiller_kølle, (ikke_spiller_køllex, ikke_spiller_kølley))
        overflate.blit(Pingpongball, (Pingpongballx, Pingpongbally))
        overflate.blit(menu_knapp, (menu_knappx, menu_knappy))
        skriv(f"Poeng: {poeng}", 50, 235, 20, WHITE) #skriver poeng og dens verdi i et bestemt posisjon 
        pygame.display.update()
        clock.tick(60) #makser spillets fps på 60 
        pos = pygame.mouse.get_pos() #henter inn informasjon om musens posisjon 
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT: #ser om spilleren avslutter programmet 
                pygame.quit() 
                sys.exit() 
            
            if event.type == pygame.MOUSEBUTTONDOWN:  #kjekker om spilleren har venstre klikket 
                if menu_knapp_rect.collidepoint(event.pos): #kjekker om mussen er innen rektangelen for menu knappen 
                    poeng = 101
                    kjøre = False
                    startside() #kjører funksjonen startside som tar spilleren tilbake til menuen 
                    return
        knap = pygame.key.get_pressed()
        #endrer posisjonen på spilelr racketen på y aksen når man trykker på opp og ned pill og også om spiller racketen ikke er på grensen av pygame vinduet 
        if knap[pygame.K_DOWN] and spiller_objekty < racket_bunn_grense:
            spiller_objekty += 5
        if knap[pygame.K_UP] and spiller_objekty > racket_topp_grense:
            spiller_objekty -= 5

        #bevegelsen av motstander racketen 
        ikke_spiller_kølley += y_akse 
        if ikke_spiller_kølley >= racket_bunn_grense or ikke_spiller_kølley <= racket_topp_grense:
            y_akse = -y_akse
            
        #bevegelsen av pingpongball
        Pingpongballx += x_akse
        Pingpongbally += ball_y_akse
        if Pingpongbally <= 0 or Pingpongbally >= 388:
            ball_y_akse = -ball_y_akse
            
        #sprette mekanik når ballen er i samme posisjon som spiller racketen 
        if (Pingpongballx <= spiller_objektx + 80 and 
            spiller_objekty < Pingpongbally < spiller_objekty + 100):
            x_akse = 5
        #sprette mekanik når ballen er i samme posisjon som motstander racketen
        if (Pingpongballx >= ikke_spiller_køllex - 20 and
            ikke_spiller_kølley < Pingpongbally < ikke_spiller_kølley + 100):
            x_akse = -5
        
        #poeng system (Øker når ballen ligger på et høyere punkt i x aksen enn motstander racketen)
        if Pingpongballx >= ikke_spiller_køllex:
            tid = pygame.time.get_ticks()
            if tid - poeng_tid > 3000:  # Vent 3s før ny poeng
                poeng += 1
                poeng_tid = tid
        
        #åpner seier vindu etter 11 poeng 
        if poeng == 11:
            win()
            return
        #åpner nederlag vindu når ballen har passert spiller racketen 
        if Pingpongballx <= spiller_objektx - 80:
            tap()
            return

#definerer nederlag vindu 
def tap():
    dutapte = pygame.image.load("playagain.png") 
    dutaptex = 140
    dutaptey = 8
    
    yes = pygame.image.load("yes.png")
    yesx = 395
    yesy = 200
    yes_rect = yes.get_rect(topleft=(395, 200))
    
    no = pygame.image.load("no.png")
    nox = 95
    noy = 200
    no_rect = no.get_rect(topleft=(95, 200))
    
    menu_knapp_stor = pygame.image.load("menuknappbig.png") 
    menu_knapp_storx = 215
    menu_knapp_story = 300
    menu_knapp_stor_rect = menu_knapp_stor.get_rect(topleft=(215, 300))
    
    taper = True 
    while taper == True:
        #setter opp pygame bakgrunn og figurer
        pygame.display.set_caption("taper")
        overflate.blit(pygame.image.load("PingPong.png"), (0, 0))
        overflate.blit(dutapte, (dutaptex, dutaptey))
        overflate.blit(yes, (yesx, yesy))
        overflate.blit(no, (nox, noy))
        overflate.blit(menu_knapp_stor, menu_knapp_stor_rect)
        pygame.display.update()
        #kjekker om spilleren har vesntre trykket 
        for event in pygame.event.get():
            if event.type == pygame.MOUSEBUTTONDOWN:
                #åpner spill vindu
                if yes_rect.collidepoint(event.pos):
                    Play()
                    return 
                #avslutter pogramm
                if no_rect.collidepoint(event.pos):
                    pygame.quit()
                    sys.exit()
                #åpner menu vindu
                if menu_knapp_stor_rect.collidepoint(event.pos):
                    startside()
                    return 
        
def win():
    duvant = pygame.image.load("youwin1.png")
    duvantx = 70
    duvanty = 8
    
    menu_knapp_stor = pygame.image.load("menuknappbig.png") 
    menu_knapp_storx = 215
    menu_knapp_story = 108
    menu_knapp_stor_rect = menu_knapp_stor.get_rect(topleft=(215, 108))
    
    seier = True
    while seier == True:
        pygame.display.set_caption("seier")
        overflate.blit(pygame.image.load("PingPong.png"), (0, 0))
        overflate.blit(help_knapp, help_knapp_rect)
        overflate.blit(duvant, (duvantx, duvanty))
        overflate.blit(menu_knapp_stor, menu_knapp_stor_rect)
        overflate.blit(close_knapp, close_knapp_rect)
        pygame.display.update() 
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                
                #åpner help vindu
                if help_knapp_rect.collidepoint(event.pos):
                    help()
                    return 
                #åpner menu vindu
                if menu_knapp_stor_rect.collidepoint(event.pos):
                    startside()
                    return 
                #avslutter programmet
                if close_knapp_rect.collidepoint(event.pos):
                    pygame.quit()
                    sys.exit()
        pygame.display.update() 


def help():
    grønn = True
    while grønn == True:
        pygame.display.set_caption("Help")
        menu_knapp = pygame.image.load("menuknapp.png")
        menu_knappx = 10
        menu_knappy = 10
        menu_knapp_rect = menu_knapp.get_rect(topleft=(10, 10))
        
        Help_opp = pygame.image.load("HelpOpp.png")
        Help_oppx = 210
        Help_oppy = 130
        
        Help_Ned = pygame.image.load("HelpNed.png")
        Help_Nedx = 210
        Help_Nedy = 230
        
        Help_Close = pygame.image.load("HelpClose.png")
        Help_Closex = 110
        Help_Closey = 300
        Help_Close_rect = Help_Close.get_rect(topleft=(110, 300))
        
        Hvordanvinne = pygame.image.load("Hvordanvinne.png")
        Hvordanvinnex = 50
        Hvordanvinney = 180
        
        Help_Play = pygame.image.load("HelpPlay.png")
        Help_Playx = 60
        Help_PLayy = 50
        Help_Play_rect = Help_Play.get_rect(topleft=(60, 50))
        
        overflate.blit(pygame.image.load("PingPong.png"), (0, 0))
        overflate.blit(menu_knapp, (menu_knappx, menu_knappy))
        overflate.blit(Help_opp, (Help_oppx, Help_oppy))
        overflate.blit(Help_Ned, (Help_Nedx, Help_Nedy))
        overflate.blit(Help_Close, (Help_Closex, Help_Closey))
        overflate.blit(Help_Play, (Help_Playx, Help_PLayy))
        overflate.blit(Hvordanvinne, (Hvordanvinnex, Hvordanvinney))
        pygame.display.update() 
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                
                #åpner menu siden
                if menu_knapp_rect.collidepoint(event.pos):
                    startside()
                    return 
                #starter spillet
                if Help_Play_rect.collidepoint(event.pos):
                    Play()
                    return 
                #avslutter programmet
                if Help_Close_rect.collidepoint(event.pos):
                    pygame.quit()
                    sys.exit()
        

def startside():
    menu = True
    while menu:
        overflate.blit(bg, (0, 0))
        overflate.blit(menu_title, menu_title_rect)
        overflate.blit(play_knapp, play_knapp_rect)
        overflate.blit(help_knapp, help_knapp_rect)
        overflate.blit(close_knapp, close_knapp_rect)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                
                #starter selve spillet
            if event.type == pygame.MOUSEBUTTONDOWN:
                if play_knapp_rect.collidepoint(event.pos):
                    Play()
                    menu = False
                    return 
                    #avslutter programmet
                if close_knapp_rect.collidepoint(event.pos):
                    pygame.quit()
                    sys.exit()
                    menu = False
                    return
                    #åpner help siden 
                if help_knapp_rect.collidepoint(event.pos):
                    help()
                    menu = False
                    return 
        pygame.display.update()
        
while menu == True:
    startside()

SystemExit: 